# Obtención de insights a partir de visualizaciones
## Descripción

Entrenarás y optimizarás diversos modelos de Machine Learning para resolver una problemática específica, detectada en la instancia de entrega anterior. El objetivo es que puedan utilizar modelos de Machine Learning para resolver el problema de una industria o negocio.

## Objetivos generales

Obtener datos de diversas fuentes como APIs o Bases de datos públicas para luego analizarlos mediante el lenguaje Python con el fin de contestar una pregunta de interés para una industria, negocio o proyecto personal. Se deberán utilizar datasets complejos implementando técnicas avanzadas para la limpieza y adquisición de datos

## Objetivos específicos

* Estructurar un problema en función de múltiples pero simples preguntas/hipótesis a responder

* Importar datos crudos de APIs o bases de datos usando Python

* Limpiar y transformar los datos para permitir un posterior análisis

* Contar una historia mediante el análisis exploratorio de datos

## Requisitos base

* **Un notebook (Colab o Jupyter) que debe contener:**
    * **Abstracto con motivación y audiencia:** Descripción de alto nivel de lo que motiva a analizar los datos elegidos y que audiencia se podrá beneficiar de este análisis

    * **Preguntas/hipótesis que queremos responder:** Lista de preguntas que se busca responder mediante el análisis de datos. Bloques de código donde se importan los datos desde una API o base de datos pública y los guarda en un archivo local csv o json. El estudiante puede luego de descargar los datos, comentar este bloque de código

    * **Análisis exploratorio de datos (EDA):** Análisis descriptivo de los datos mediante visualizaciones y herramientas estadísticas.


**Una presentación (PDF; PowerPoint o Google Slides) que debe contener:**
* Abstracto con motivación y audiencia 
* Descripción de alto nivel de lo que motiva a analizar los datos elegidos y que audiencia se podrá beneficiar de este análisis

* Resumen de metadata: resumen de los datos a ser analizados es decir, número de filas/columnas, tipos de variables, etc

* Preguntas hipótesis que queremos responder: Lista de preguntas que se busca responder mediante el análisis de datos

* Visualizaciones ejecutivas que responden nuestras preguntas: utilización de gráficos que responden las preguntas de interés de nuestro proyecto.

* Insights: resumen de hallazgos del proyecto. Aquí consolidamos las respuestas a las preguntas/hipótesis que fuimos contestando con las visualizaciones


## Sugerencias

Es conveniente retomar el dataset trabajado en la primera pre entrega y enriquecerlo (e.g joins, y creación de nuevas columnas) con información proveniente de APIs públicas siempre que se pueda con el fin de practicar las nuevas habilidades adquiridas. Se recomienda retomar la metodología de trabajo y reutilizar algoritmos ya entrenados, de ser necesario.

## Requisitos extra

* Subir el proyecto a Github

## Dont’s

* Utilizar jerga demasiado técnica en la presentación (recordar que la audiencia de la misma son roles ejecutivos)

* Sobrecargar las diapositivas

* Realizar una presentación con más de 12 slides de extensión

# Entrega 1

## 